In [1]:
#!pip install opencv-python pytesseract googlesearch-python
import os
import cv2
import pytesseract
from googlesearch import search
import csv

#Ruta de Tesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

#Funcion: Itera sobre imágenes en un directorio, selecciona un área de interés, extrae el texto, realiza una búsqueda en Google con el texto y guarda los resultados en un archivo CSV.
def seleccionar_area_extraer_texto_y_buscar(directorio, escala=0.5, archivo_salida="resultados.csv"):
    with open(archivo_salida, mode="w", newline="", encoding="utf-8") as archivo_csv:
        escritor_csv = csv.writer(archivo_csv)
        escritor_csv.writerow(["Nombre de la Imagen", "Texto Extraído", "URL 1", "URL 2", "URL 3"])
        for archivo in os.listdir(directorio): #Iterar sobre archivos en el directorio
            ruta_completa = os.path.join(directorio, archivo) #Ruta completa del archivo
            if os.path.isfile(ruta_completa) and archivo.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')): #Verificar que sea imagen
                imagen = cv2.imread(ruta_completa) #Leer la imagen
                if imagen is None:
                    print(f"Error al cargar la imagen: {archivo}")
                    continue
                print(f"Procesando: {archivo}")
                #Zoom out
                altura_original, ancho_original = imagen.shape[:2] #Dimensiones originales
                nuevo_ancho = int(ancho_original * escala) #Nuevo ancho
                nueva_altura = int(altura_original * escala) #Nueva altura
                imagen_redimensionada = cv2.resize(imagen, (nuevo_ancho, nueva_altura)) #Redimensionar
                
                roi = cv2.selectROI("Selecciona un área y presiona ENTER", imagen_redimensionada) #Devuelve (x, y, w, h)
                cv2.destroyWindow("Selecciona un área y presiona ENTER") #Cerrar la ventana
                if roi == (0, 0, 0, 0): #Si no se selecciona un área válida
                    print("No se seleccionó ningún área. Pasando a la siguiente imagen.")
                    continue

                # Mapear el ROI a las dimensiones originales
                x, y, w, h = roi
                x_original = int(x / escala)
                y_original = int(y / escala)
                w_original = int(w / escala)
                h_original = int(h / escala)

                # Recortar el área seleccionada
                area_interes = imagen[y_original:y_original + h_original, x_original:x_original + w_original] #Extraer la región seleccionada
                
                # Extraer texto del área seleccionada con Tesseract
                try:
                    texto_extraido = pytesseract.image_to_string(area_interes, lang="eng") #Cambiar "eng" si usas otro idioma
                    if not texto_extraido.strip(): #Si no se extrajo texto
                        texto_extraido = "Error en OCR"
                except Exception as e:
                    texto_extraido = "Error en OCR"
                    print(f"Error en OCR para {archivo}: {e}")
                
                print(f"Texto extraído de {archivo}:\n{texto_extraido}\n")

                # Realizar búsqueda en Google con el texto extraído
                resultados = []
                try:
                    for url in search(texto_extraido, num_results=3): #Obtener los 3 primeros resultados
                        resultados.append(url)
                except Exception as e:
                    print(f"Error en la búsqueda para {archivo}: {e}")
                    resultados = ["Error en búsqueda"] * 3

                # Asegurar que hay 3 resultados, incluso si hay errores
                while len(resultados) < 3:
                    resultados.append("")

                # Escribir los datos en el archivo CSV
                escritor_csv.writerow([archivo, texto_extraido] + resultados)
                print(f"Resultados guardados para {archivo}.")

# Ruta del directorio que contiene las imágenes
directorio_imagenes = "." #Reemplaza con tu directorio

# Ejecutar la función
seleccionar_area_extraer_texto_y_buscar(directorio_imagenes, escala=0.2, archivo_salida="resultados.csv")

Procesando: IMG_20250120_202417325.jpg
Texto extraído de IMG_20250120_202417325.jpg:
REMOTE SENSING AS A LANDSCAPE EPIDEMIOLOGIC TOOL TO
IDENTIFY VILLAGES AT HIGH RISK
FOR MALARIA TRANSMISSION
LOUISA R. BECK, MARIO H. RODRIGUEZ, SHERI W. DISTER, AMERICO D.
RODRIGUEZ, ELISKA REJMANKOVA, ARMANDO ULLOA. ROSA A. MEZA. DONALD

R. ROBERTS. JACK F. PARIS. MICHAEL A. SPANNER. ROBERT K. WASHINO.
CARL HACKER, anp LLEWELLYN J. LEGTERS


Resultados guardados para IMG_20250120_202417325.jpg.
Procesando: IMG_20250120_202434412.jpg
Texto extraído de IMG_20250120_202434412.jpg:
ASSESSMENT OF A REMOTE SENSING-BASED MODEL FOR PREDICTING MALARIA
TRANSMISSION RISK IN VILLAGES OF CHIAPAS, MEXICO

LOUISA R. BECK, MARIO H. RODRIGUEZ, SHERI W. DISTER, AMERICO D. RODRIGUEZ, ROBERT K. WASHINO,
DONALD R. ROBERTS, anpD MICHAEL A. SPANNER


Resultados guardados para IMG_20250120_202434412.jpg.
Procesando: IMG_20250120_202449824.jpg
Texto extraído de IMG_20250120_202449824.jpg:
THE VIRUSES

Biochemical, Biological,